# Assignment10_HCordoba

## First we are going to export all the assignment 9 in order to start with Assignment 10

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

### Importing the external files

In [ ]:
ExternalFilesFolder = r"C:\Users\gelli\Desktop\POLIMI FIRST YEAR\ENERGY AND ENVIRONMENTAL SYSTEM FOR BUILDINGS\Files solved in Python\Week 9"
ConsumptionFileName= "consumption_5545.csv"
TemperatureFileName= "Austin_weather_2014.csv"
IrradianceFileName= "irradiance_2014_gen.csv"

path_consumptionFile = os.path.join(ExternalFilesFolder,ConsumptionFileName)
path_TemperatureFile = os.path.join(ExternalFilesFolder,TemperatureFileName)
path_IrradianceFile = os.path.join(ExternalFilesFolder,IrradianceFileName)

### Lets import all our commands from Assignment9

In [ ]:
DF_consumption = pd.read_csv(path_consumptionFile,sep=",", index_col=0)
DF_consumption.head()
DF_consumption.tail(10)


#We transform the data in the file from 'undefined objects' to 'time and date data'
PreviousIndex = DF_consumption.index
NewParsedIndex= pd.to_datetime(PreviousIndex)
DF_consumption.index =NewParsedIndex 

DF_consumption.head()
DF_consumption.index.hour
DF_consumption.index.month
DF_consumption.index.dayofweek


#We extract/select the data we are interested in: in this case from 2014-07-01 00:00:00 to 2014-07-03 23:00:00
DF_consumption_someDaysInJuly=DF_consumption["2014-07-01 00:00:00":"2014-07-03 23:00:00"]


#We plot the AC Power (in W) consumption versus time during the period we are considering.
plt.figure()
plt.plot(DF_consumption_someDaysInJuly)
plt.xlabel("Time")
plt.ylabel("AC Power (W)")
plt.show()


#This is a second way of doing it (better than the other):
plt.figure()
DF_consumption_someDaysInJuly.plot()
plt.xlabel("Time")
plt.ylabel("AC Power (W)")
plt.show()


#Now we import the weather data, specifically temperature
DF_weather = pd.read_csv(path_TemperatureFile,sep=";",index_col=0)
DF_weather.head(24)

#We transform the data in the file from 'undefined objects' to 'time and date data'
previousIndex_weather=DF_weather.index
newIndex_weather=pd.to_datetime(previousIndex_weather)
DF_weather.index = newIndex_weather
DF_weather.columns
Series_Temperature = DF_weather["temperature"]
DF_Temperature= DF_weather[["temperature"]]
DF_Temperature.head()


#Now we import the weather data, specifically irradiance
DF_irradianceSource = pd.read_csv(path_IrradianceFile,sep=";",index_col=1)
DF_irradianceSource.head(24)
DF_irradiance=DF_irradianceSource[["gen"]]
DF_irradiance.head(24)


#We change the generated irradiance less than 0 into 0 value
DF_irradiance["gen"]<0
DF_irradiance[DF_irradiance["gen"]<0] = 0
DF_irradiance.head(24)


#We join the weather data in the same array
DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])
DF_joined.head(24)

#We remove missing values using .dropna()
DF_joined_cleaned = DF_joined.dropna()
DF_joined_cleaned.head(24)


#We have the pick of temperature that was a 7h which is wrong. The reason is that we are using the time GTM from Greenwhich handling data from Texas. So, we change it just for one day, normalizing as follows:
DF_joined_cleaned_copy = DF_joined.dropna().copy()
DF_joined_cleaned_chosenDates = DF_joined_cleaned_copy["2014-08-01":"2014-08-04"]

#Solving the problem with the timezone
DF_joined_cleaned_chosenDates["temperature"]=DF_joined_cleaned_chosenDates["temperature"].shift(-5)
DF_joined_cleaned_chosenDates.dropna()
DF_joined_cleaned_chosenDates.head()
DF_joined_cleaned_chosenDates.describe()

DF_joined_cleaned_chosenDates_min=DF_joined_cleaned_chosenDates.min()
DF_joined_cleaned_chosenDates_max=DF_joined_cleaned_chosenDates.max()
DF_joined_cleaned_chosenDates_normalized= (DF_joined_cleaned_chosenDates-DF_joined_cleaned_chosenDates_min)/(DF_joined_cleaned_chosenDates_max-DF_joined_cleaned_chosenDates_min)
plt.figure()
DF_joined_cleaned_chosenDates_normalized.plot()


#Now we want to create lagged features.
#First, we reconstruct the dataframe by appliying time-zone
DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])
DF_mod = DF_joined.copy()

#We have a delay of 5 hours because of time-zoning. We correct it:
DF_mod["temperature"]=DF_mod["temperature"].shift(-5)
DF_mod.dropna(inplace=True) # It is the same as writing it like this: DF_mod=DF_mod.dropna()

#In order to consider the time delays, we add .shift() in temperature array:
#DF_mod.head()
#DF_mod.describe()

#DF_mod["Temperature -1h"]= DF_mod["temperature"].shift(1)
#DF_mod["Temperature -2h"]= DF_mod["temperature"].shift(2)
#DF_mod["Temperature -3h"]= DF_mod["temperature"].shift(3)
#DF_mod["Temperature -4h"]= DF_mod["temperature"].shift(4)
#DF_mod["Temperature -5h"]= DF_mod["temperature"].shift(5)
#DF_mod["Temperature -6h"]= DF_mod["temperature"].shift(6)
#DF_mod.head()

#A better way to consider and solve the time delays with the other parameters (such as irradiance and AC Power consumption) is creating a function that includes a bucle for:
#First we create a range, setting the array lenght
lag_start=1
lag_end = 6
lag_interval=1
column_name="temperature"
df=DF_mod

#Then, we create the bucle
for i in range(lag_start,lag_end+1,lag_interval):
    new_column_name = column_name+" -"+str(i)+"hr"
    print new_column_name
    df[new_column_name]=df[column_name].shift(i)   
    df.dropna(inplace=True) #this removes all the row with a Nan


#And finally we create the function including for and .shift() commands
def lag_feature(df,column_name,lag_start,lag_end,lag_interval):
    for i in range(lag_start,lag_end+1,lag_interval):
        new_column_name = column_name+" -"+str(i)+"hr"
        print new_column_name
        df[new_column_name]=df[column_name].shift(i)   
        df.dropna(inplace=True) #this removes all the row with a Nan
    return df


#We check it works with and example
DF_modified=lag_feature(DF_mod,"temperature",1,6,1)
DF_modified.head()

#We do the same for the irradiance and consumption
#We renaming the column names, there are two ways:

#DF_mod.columns =["AC_consumption","temperature","irradiance"]
#The second way of doing this:
DF_mod=DF_mod.rename(columns={"air conditioner_5545":"AC_consumption","gen":"irradiance"})

#We use the function for temperature, just for 1 to 6 hours:
DF_mod= lag_feature(DF_mod,"temperature",1,6,1)
DF_mod.head()

#We use the function for irradiance, just for 3 to 6 hours:
DF_mod= lag_feature(DF_mod,"irradiance",3,6,1)

#We add the previous consumptions in the last 24 hours!
DF_mod=  lag_feature(DF_mod,"AC_consumption",1,24,1)
DF_mod.head()
DF_mod.describe()

#Now we add the seasonality parameters (time-related parameters)
DF_mod["hour"]=DF_mod.index.hour
DF_mod["hour"].head()
DF_mod["sin_hour"]=np.sin(DF_mod.index.hour*2*np.pi/24)
DF_mod["cos_hour"]=np.cos(DF_mod.index.hour*2*np.pi/24)

DF_mod["day_of_week"]=DF_mod.index.dayofweek
DF_mod[["hour","sin_hour","cos_hour","day_of_week"]].head(24)

DF_mod["month"]=DF_mod.index.month

DF_mod["week_of_year"]=DF_mod.index.week


#We create a function that recognises weekends:
def WeekendDetector(day):
    if (day==5 or day == 6):
        weekendLabel=1
    else:
        weekendLabel=0
    return weekendLabel


#And we apply the function with an example
DF_mod["weekend"]= DF_mod["day_of_week"].apply(WeekendDetector)

#We create a function that detects working time between 9am-7pm hours  
def DayDetector(hour):
    if (hour< 19 and hour>=9):
        DayLabel=1
    else:
        DayLabel=0
    return DayLab



# Now let's start with Assignment #10

## Machine Learning

In [ ]:
DF_mod.head()
DF_mod.columns
DF_mod.corr()
DF_mod.info()

#### Now we choose our data frame to be analyzed on the date where we are going to use most the air conditioning

In [ ]:
DF_mod=DF_mod["2014-03-01":"2014-09-30"]

#### We define our target, that is the main variable that we are going to analyze or to predict

In [ ]:
DF_target=DF_mod["AC_consumption"]

#### We define our features but we remove the axis 1 with function .drop

In [ ]:
DF_features=DF_mod.drop("AC_consumption",axis=1)

#### Now we divide between train and test subgroups. We add our variables to manage and also we add a random state in order that everybody have the same random values
#### X_train and Y_train are the random values that we are going to use for training, X corresponds to features and Y corresponds to target
#### X_test and Y_test are the random values that we are going to use for predict, X corresponds to features and Y corresponds to target

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(DF_features,DF_target,test_size=0.2,random_state=41234)

#### Now We start using models

In [ ]:
from sklearn import linear_model
linear_reg=linear_model.LinearRegression()

#### The second step will be fitting a model. We add a variable that will predict the model of X test that are the features. Then we apply this prediction to a dataframe and with add the Ytest in order to compare the features with the target.

In [ ]:
linear_reg.fit(X_train,Y_train)

predicted_linearReg_split=linear_reg.predict(X_test)
predicted_DF_linearReg_split=pd.DataFrame(predicted_linearReg_split,index=Y_test.index,columns=["AC_cons_predicted_linear_reg"]) 
predicted_DF_linearReg_split=predicted_DF_linearReg_split.join(Y_test) 

#### We uses this to check if the prediction is working, and we select the month of august.

In [ ]:
predicted_DF_linearReg_split_august=predicted_DF_linearReg_split["2014-08-01":"2014-08-31"]

#### let's plot to see how similar could be the prediction to the real values

In [ ]:
predicted_DF_linearReg_split_august.plot()

#### Now we want to estimate how accurate are our predictions and we use  MAE, MSE, R2 functions of prediction. Again we import everything.

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
    

#### Measure of difference between two continuous variables

In [ ]:
MAE_linearReg_split=mean_absolute_error(predicted_linearReg_split,Y_test) 

#### Measures the average of the squares of the errors

In [ ]:
MSE_linearReg_split=mean_squared_error(predicted_linearReg_split,Y_test)

#### Coefficient of determination regression score function.

In [ ]:
R2_linearReg_split=r2_score(predicted_linearReg_split,Y_test)

#### The second way of doing this is using k-folf cross validation
##### cross validation is another way of predicting learning to compare and select a model for a given predictive modeling problem 
#### We follow the same steps that we did for split but with other method

In [ ]:
from sklearn.model_selection import cross_val_predict
predicted_linearReg_CV=cross_val_predict(linear_reg,DF_features,DF_target,cv=10) #cv=10 porque dividimos en 10 subgrupos
predicted_DF_linearReg_CV=pd.DataFrame(predicted_linearReg_CV,index=DF_target.index,columns=["AC_cons_predicted_linear_CV"])
predicted_DF_linearReg_CV=predicted_DF_linearReg_CV.join(DF_target) #.join() matches index = matches the data

predicted_DF_linearReg_CV_august=predicted_DF_linearReg_CV["2014-08-01":"2014-08-31"]
predicted_DF_linearReg_CV_august.plot()

#### Now we calculate again how accurate where the predictions with this new method

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

MAE_linearReg_CV=mean_absolute_error(predicted_linearReg_CV,DF_target)
MSE_linearReg_CV=mean_squared_error(predicted_linearReg_CV,DF_target)
R2_linearReg_CV=r2_score(predicted_linearReg_CV,DF_target)

#### Now we try other algorithm
#### The third way of doing this is using RandomForestRegressor
##### meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging and select a model for a given predictive modeling problem 
#### We follow the same steps that we did for split but with other method

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg_RF=RandomForestRegressor()

predicted_RF_CV=cross_val_predict(reg_RF,DF_features,DF_target,cv=10) #cv=10 porque dividimos en 10 subgrupos
predicted_DF_RF_CV=pd.DataFrame(predicted_RF_CV,index=DF_target.index,columns=["AC_cons_predicted_RF_CV"])
predicted_DF_RF_CV=predicted_DF_RF_CV.join(DF_target) #.join() matches index = matches the data

predicted_DF_RF_CV_august=predicted_DF_RF_CV["2014-08-01":"2014-08-31"]
predicted_DF_RF_CV_august.plot()

MAE_RF_CV=mean_absolute_error(predicted_RF_CV,DF_target)
MSE_RF_CV=mean_squared_error(predicted_RF_CV,DF_target)
R2_RF_CV=r2_score(predicted_RF_CV,DF_target)

#### If we want to use online learning.We take our data train

In [ ]:
DF_onlineConsumptionPrediction=pd.DataFrame(index=DF_mod.index)

#### We select a period of training that would be 30 days

In [ ]:
period_of_training=pd.Timedelta(30,unit="d")

#### With this variable, we apply the first date where we are going to start the model for AC consumption

In [ ]:
FirstTimeStamp_measured=DF_mod.index[0]

#### This variable represent the last date of your model for AC consumption

In [ ]:
LastTimeStamp_measured=DF_mod.index[-1]

#### Firsttime to predict is going to be after 30 days of the starting day that is our firstTimeStamp_measured

In [ ]:
FirstTimeStamp_toPredict=FirstTimeStamp_measured+period_of_training

#### Our first day to training is going to start in our first date for our model

In [ ]:
training_startTimeStamp=FirstTimeStamp_measured

#### Our training is going to finish when it reaches this 30 days of training for every month

In [ ]:
training_endTimeStamp=FirstTimeStamp_toPredict

#### The first day to predict is going to start when the training ends

In [ ]:
timeStamp_toPredict=FirstTimeStamp_toPredict


#### Before training_endTimeStamp, remove everything for the consumption prediction

In [ ]:
DF_onlineConsumptionPrediction.truncate(before=training_endTimeStamp)  

#### Then we are going to do a condition for our model, between training and prediction With this while, we can use any of the previous model to do the predictions, but with the while instead of changing and predicted for days
#### We are training first for the 30 days and the we are predicting every hour, so we choose the data of the previous month and then we predict the next hour. For doing this procedures of taking the previous as training and the predict the next hour, 
#### We are changing the variable by adding them a timedelta of one hour, so everytime, we are going to change our training and test for our features and also for our target and all this loop is starting from 1st of March until he finnally recognize that we are in the end of.Our model that is the last day of the month of september.

#### Useful to use the data of the previous month because. It is good to predict and also because is recent so it is better to do it one month before

In [ ]:
while (timeStamp_toPredict<LastTimeStamp_measured):
    
    DF_feature_train=DF_features.truncate(before=training_startTimeStamp,after=training_endTimeStamp) #.truncate() elimina todo lo que no esté comprendido entre los valores before-after
    DF_target_train=DF_target.truncate(before=training_startTimeStamp,after=training_endTimeStamp)
    DF_feature_test=DF_features.loc[timeStamp_toPredict].values.reshape(1,-1) #we need to add it, otherwise it gives error
    DF_target_test=DF_target.loc[timeStamp_toPredict]
    reg_RF.fit(DF_feature_train,DF_target_train) #this is for traininig with the training data
    predicted_Consumption=reg_RF.predict(DF_feature_test)
    DF_onlineConsumptionPrediction.loc[timeStamp_toPredict,"Predicted"]=predicted_Consumption
    DF_onlineConsumptionPrediction.loc[timeStamp_toPredict,"Real"]=DF_target_test

    timeStamp_toPredict=timeStamp_toPredict+pd.Timedelta(1,unit="h") 
    training_endTimeStamp=training_endTimeStamp+pd.Timedelta(1,unit="h")
    training_startTimeStamp=training_startTimeStamp+pd.Timedelta(1,unit="h")


DF_onlineConsumptionPrediction.dropna(inplace=True)

#### So we calculate how accurate is with the online learning for model prediction Random Forest

In [ ]:
R2_score_online_RF=r2_score(DF_onlineConsumptionPrediction[["Real"]],DF_onlineConsumptionPrediction[["Predicted"]])